<a href="https://colab.research.google.com/github/Shashank-Ojha/WordFormer/blob/main/trainer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
!pip install torch
%load_ext autoreload
%autoreload 2

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
from utils.attention_block import foo_bar

foo_bar()

tensor([[0.5666, 0.0087, 0.3581, 0.4695, 0.1558],
        [0.1622, 0.2578, 0.2463, 0.5294, 0.4225],
        [0.8683, 0.1375, 0.8439, 0.8943, 0.2691],
        [0.2155, 0.3078, 0.3846, 0.6624, 0.7329],
        [0.0522, 0.9557, 0.4536, 0.0911, 0.2918]])
